In [1]:
!pip install optuna

  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=ffaf7a4f4bceb16877fb72bcfcbe2c803d484db3843ce8037118c9eaaeca0f63
  Stored in directory: c:\users\muhammad hamza abid\appdata\local\pip\cache\wheels\0c\09\9e\49e21a6840ef7955b06d47394afef0058f0378c0914e48b8b8
Successfully built pyperclip


In [2]:
import optuna

In [3]:
!pip install xgboost

In [2]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [8]:
df = pd.read_csv('Admission_Prediction.csv')
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332.0,108.0,5.0,4.5,4.0,9.02,1,0.87
496,497,337.0,117.0,5.0,5.0,5.0,9.87,1,0.96
497,498,330.0,120.0,5.0,4.5,5.0,9.56,1,0.93
498,499,312.0,103.0,4.0,4.0,5.0,8.43,0,0.73


In [9]:
x = df.drop(columns = ['Serial No.', 'Chance of Admit'])

In [10]:
x

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.0,118.0,4.0,4.5,4.5,9.65,1
1,324.0,107.0,4.0,4.0,4.5,8.87,1
2,NaN,104.0,3.0,3.0,3.5,8.00,1
3,322.0,110.0,3.0,3.5,2.5,8.67,1
4,314.0,103.0,2.0,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332.0,108.0,5.0,4.5,4.0,9.02,1
496,337.0,117.0,5.0,5.0,5.0,9.87,1
497,330.0,120.0,5.0,4.5,5.0,9.56,1
498,312.0,103.0,4.0,4.0,5.0,8.43,0


In [11]:
y = df['Chance of Admit']
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

In [20]:
from sklearn.metrics import mean_squared_error   
def Objective(trial, data = x, target = y):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size = 0.1, random_state = 30)
    param = {
        'tree_method':'gpu_hist',
        'lambda':trial.suggest_loguniform('lambda', 1e-4,10.0),
        'alpha':trial.suggest_loguniform('alpha', 1e-4,10.0),
        'colsample_bytree':trial.suggest_categorical('colsample_bytree', [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'subsample':trial.suggest_categorical('sub_sample', [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'learning_rate': trial.suggest_categorical('learning_rate', [.0001, .0003, .0008, .02,.01,1,10,20]),
        'n_estimators': 30000,
        'max_depth':trial.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
        'random_state': trial.suggest_categorical('random_state', [10,20,30, 2000,3454,243123]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,200),
        'verbosity': 3
    }
    xgb_reg_model = xgb.XGBRFRegressor(**param)
    xgb_reg_model.fit(train_x, train_y, eval_set = [(test_x, test_y)], verbose = True)
    pred_xgb = xgb_reg_model.predict(test_x)
    rmse = mean_squared_error(test_y,perd_xgb)
    return rmse

In [21]:
find_param = optuna.create_study(direction = 'minimize')
find_param.optimize(Objective, n_trials = 10)
find_param.best_params 

[I 2022-06-28 00:09:48,595] A new study created in memory with name: no-name-87ab1404-284d-4bf2-958b-b728ef678fc5
[W 2022-06-28 00:09:48,659] Trial 0 failed because of the following error: XGBoostError('[00:09:48] C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/gbm/gbtree.cc:597: Check failed: common::AllVisibleGPUs() >= 1 (0 vs. 1) : No visible GPU is found for XGBoost.')
Traceback (most recent call last):
  File "C:\Users\Muhammad Hamza Abid\anaconda3\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\MUHAMM~1\AppData\Local\Temp/ipykernel_7116/174706176.py", line 18, in Objective
    xgb_reg_model.fit(train_x, train_y, eval_set = [(test_x, test_y)], verbose = True)
  File "C:\Users\Muhammad Hamza Abid\anaconda3\lib\site-packages\xgboost\core.py", line 532, in inner_f
    return f(**kwargs)
  File "C:\Users\Muhammad Hamza Abid\anaconda3\lib\site-packages\xgboost\sklearn.py", line 1649, in fit
    su

[00:09:48] DEBUG: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_gpu_hist.cu:817: [GPU Hist]: Configure
[00:09:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:248: No visible GPU is found, setting `gpu_id` to -1


XGBoostError: [00:09:48] C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/gbm/gbtree.cc:597: Check failed: common::AllVisibleGPUs() >= 1 (0 vs. 1) : No visible GPU is found for XGBoost.

In [22]:
!NVIDIA-SMI

'NVIDIA-SMI' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
# Classification

In [10]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna

In [5]:
df_1 = pd.read_csv('https://raw.githubusercontent.com/aniruddhachoudhury/Red-Wine-Quality/master/winequality-red.csv')
df_1

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [6]:
x = df_1.drop(columns = 'quality')
y = df_1.quality

In [7]:
x

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


In [8]:
def objective_classification(trial):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
    param = {
        'tree_method':'gpu_hist',
        'verbosity': 3,
        'objective':'binary:logistics',
        'booster': trial.suggest_categorical('booster', ['dart', 'gbtree', 'gblinear'] ),
        'lambda':trial.suggest_float('lambda', 1e-4,1),
        'alpha':trial.suggest_float('alpha', 1e-4, 1),
        'subsample':trial.suggest_float('subsample', .1,.5),
        'colsample_bytree': trial.suggest_float('colsample_bytree', .1,.5)
    }
    if param['booster'] in ['gbtree', 'dart']:
        param['gamma']: trial.suggest_float('gamma', 1e-3, 4)
        param['eta']: trial.suggest_float('eta', .001, 5)

    xgb_classification = xgb.XGBClassifier(**param)
    xgb_classification.fit(x_train, y_train, eval_set=[x_test, y_test])
    pred = xgb_classification.predict(x_test)
    accuracy = xgb_classification.accuracy_score(x_test, y_test)

    return accuracy

In [11]:
xgb_classification_optuna = optuna.create_study(direction = 'minimize')
xgb_classification_optuna.optimize(objective_classification, n_trials = 10)

[I 2022-06-28 12:51:03,075] A new study created in memory with name: no-name-964d1822-1244-490a-85de-675c835f2f7d
[W 2022-06-28 12:51:03,120] Trial 0 failed because of the following error: ValueError('Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [3 4 5 6 7 8]')
Traceback (most recent call last):
  File "C:\Users\Muhammad Hamza Abid\anaconda3\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\MUHAMM~1\AppData\Local\Temp/ipykernel_11244/2973160523.py", line 18, in objective_classification
    xgb_classification.fit(x_train, y_train, eval_set=[x_test, y_test])
  File "C:\Users\Muhammad Hamza Abid\anaconda3\lib\site-packages\xgboost\core.py", line 532, in inner_f
    return f(**kwargs)
  File "C:\Users\Muhammad Hamza Abid\anaconda3\lib\site-packages\xgboost\sklearn.py", line 1357, in fit
    raise ValueError(
ValueError: Invalid classes inferred from unique values of `y`.  Expected: [

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [3 4 5 6 7 8]